<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import salem
import seaborn as sns
import xarray as xr
from scipy import optimize as optimization
from collections import OrderedDict
%matplotlib inline

## Errors 

In [ ]:
dfe = pd.read_csv('/home/mowglie/disk/OGGM_Output/list_errors.csv', index_col=0)

### Glacier start volume for later 

In [ ]:
dd = '/home/mowglie/disk/OGGM_Output/run_output_summary'
rgi_regs = ['rgi_reg_{:02}'.format(p) for p in np.arange(1, 19)]
df = []
for r in rgi_regs:
    p = os.path.join(dd, r, 'glacier_characteristics.csv')
    _df = pd.read_csv(p, index_col=0, low_memory=False)
    _df['rgi_reg'] = r[-2:]
    df.append(_df)
vdf = pd.concat(df)

In [ ]:
vdf = vdf.loc[~vdf.index.isin(dfe.index)]
assert np.all(~vdf.inv_volume_km3.isnull())

# Read in model output global

In [ ]:
cols = ['run_output_rdn_2000', 
        'run_output_rdn_2000_tbias_m05', 
        'run_output_rdn_2000_tbias_p05',
        'run_output_rdn_tstar',
        'run_output_rect_rdn_tstar',
       ]

In [ ]:
dd = '/home/mowglie/disk/OGGM_Output/run_output_summary'
rgi_regs = ['rgi_reg_{:02}'.format(p) for p in np.arange(1, 19)]
out = OrderedDict()
for r in rgi_regs:
    _df = pd.DataFrame()
    for c in cols:
        p = os.path.join(dd, r, c + '.nc')
        with xr.open_dataset(p) as ds:
            vol = ds.volume.load().isel(rgi_id=~ds.rgi_id.to_series().isin(dfe.index))
            vol = vol.sum(dim='rgi_id').to_series() * 1e-9
        _df[c] = vol
    out[r] = _df

In [ ]:
tdf = out[rgi_regs[0]]
for r in rgi_regs[1:]:
    tdf = tdf + out[r]

In [ ]:
(tdf * 1e9 / 3.62e14).plot();

In [ ]:
(tdf / tdf.iloc[0]).plot();

In [ ]:
(tdf / tdf.iloc[0]).iloc[-1]

### Sorted per volume

In [ ]:
cs = vdf.inv_volume_km3.sort_values().cumsum()
cs.plot();

In [ ]:
percs = np.array([1/4, 3/4]) * cs.iloc[-1]
percs

In [ ]:
ids_1 = cs.loc[cs <= percs[0]].index
ids_2 = cs.loc[(cs > percs[0]) & (cs <= percs[1])].index
ids_3 = cs.loc[cs > percs[1]].index
len(cs), len(ids_1), len(ids_2), len(ids_3)

In [ ]:
dd = '/home/mowglie/disk/OGGM_Output/run_output_summary'
rgi_regs = ['rgi_reg_{:02}'.format(p) for p in np.arange(1, 19)]

tfd_perc = []
for ids in [ids_1, ids_2, ids_3]:
    out_per_reg = OrderedDict()
    for r in rgi_regs:
        _df = pd.DataFrame()
        for c in cols:
            p = os.path.join(dd, r, c + '.nc')
            with xr.open_dataset(p) as ds:
                vol = ds.volume.load().isel(rgi_id=ds.rgi_id.to_series().isin(ids))
                vol = vol.sum(dim='rgi_id').to_series() * 1e-9
            _df[c] = vol
        out_per_reg[r] = _df
        _tdf = out_per_reg[rgi_regs[0]]
    for r in rgi_regs[1:]:
        _tdf = _tdf + out_per_reg[r]
    tfd_perc.append(_tdf)

In [ ]:
sns.set_style('whitegrid')

In [ ]:
titles = ['a: global', 'b: lower quartile', 'c: upper quartile']
fi = 0.7
f, axs = plt.subplots(1, 3, figsize=(11*fi, 4*fi), sharey=True)
letkm = dict(color='black', ha='left', va='bottom',
             bbox=dict(facecolor='w', alpha=1., edgecolor='lightgrey'))
xt, yt = 0.027, 0.91
for data, ax, t, tn in zip([tdf, tfd_perc[0], tfd_perc[-1]], axs, titles, [len(cs), len(ids_1), len(ids_3)]):
    data = data.copy()
    data.columns = ['1985-2015', '1985-2015 $-$0.5°C', '1985-2015 $+$0.5°C', 't*', 't* (rectangular)']
    
    for (c, a, col, s) in zip(data.columns, 
                 [1., 0.5, 0.5, 1, 1], 
                 ['k', 'C0', 'C3', 'rebeccapurple', 'mediumpurple'], 
                 ['-', '-', '-', '--', '--']):
         (data / data.iloc[0])[c].plot(ax=ax, legend=False, 
                                   alpha=a,
                                   color=col,
                                   style=s);           
    ax.set_xlabel('Years')
    ax.set_xlim([0, 300])
    ax.set_ylim([0.25, 1.3])
    ax.text(xt, yt, t + ' N={}'.format(tn), **letkm, transform=ax.transAxes)
    
axs[0].set_ylabel('Fraction of initial volume');
axs[2].legend(frameon=True, facecolor='white', framealpha=1, loc=(0, 0), fontsize=9);

# Save
plt.tight_layout()
plt.savefig('/home/mowglie/disk/MyProjects/Papers/2017_OGGM/gmdd/figs/global_runs.pdf', dpi=300, bbox_inches='tight')

In [ ]:
h = np.arange(300)
w = np.sqrt(4*h/0.003)
plt.plot(w)
plt.figure();
plt.plot(w[1:] - w[:-1]);

In [ ]:
from oggm.utils import parse_rgi_meta

In [ ]:
rname, _ = parse_rgi_meta(version='6')

In [ ]:
letkm = dict(color='black', ha='left', va='bottom', fontsize=9, 
             bbox=dict(facecolor='w', alpha=.5, edgecolor='lightgrey'))
xt, yt = 0.03, 0.04
fi = 0.8
f, axs = plt.subplots(6, 3, figsize=(8*fi, 12*fi), sharex=True)
axs = axs.flatten()
for i, r in enumerate(out.keys()):
    df = out[r]
    ax = axs[i]
    df = df[['run_output_rdn_2000', 'run_output_rdn_2000_tbias_m05', 'run_output_rdn_2000_tbias_p05']].copy()
    df.columns = ['1985-2015', '$-$0.5°C', '$+$0.5°C']
    
    _data = (df / 1000)
    for (c, a, col, s) in zip(_data.columns, 
             [1., 0.5, 0.5], 
             ['k', 'C0', 'C3'], 
             ['-', '-', '-']):
         _data[c].plot(ax=ax, legend=False, lw=2, 
                                   alpha=a,
                                   color=col,
                                   style=s);   
    ax.set_xlim([0, 300])
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    rc = r[-2:]
    rn = rname.loc[int(rc)].values[0]
    if 'Middle' in rn:
        rn = 'Caucasus and ME'
    rn = rn.replace(' and', ' &')
    rn = rn.replace('Southern', 'S.')
    rn = rn.replace('South', 'S.')
    rn = rn.replace('North', 'N.')
    rn = rn.replace('Western', 'W.')
    ax.text(xt, yt, rc + ': ' + rn, **letkm, transform=ax.transAxes)
    
    mlim = 0
    if rc in ['02']:
        mlim = _data.min().min()*0.7
    if rc in ['11', '16']:
        mlim = 0.02
    if rc in ['18']:
        mlim = 0.01
    
    ax.set_ylim([_data.min().min()*0.7 - mlim, _data.max().max()*1.1])
    
    
axs[0].text(-0.1, 1.1, 'Y-axis: 10$^{3}$ km$^{3}$    X-axis: years',
             color='black', ha='left', va='bottom', transform=axs[0].transAxes)
axs[1].legend(bbox_to_anchor=(1.3, 1.07), loc='lower center', ncol=3, borderaxespad=0.);
plt.tight_layout()
plt.subplots_adjust(top=0.94)
plt.savefig('/home/mowglie/disk/MyProjects/Papers/2017_OGGM/gmdd/figs/global_reg_runs.pdf', dpi=300, bbox_inches='tight')

In [ ]:
_data.min().min()

### Commited 

In [ ]:
cdf = pd.DataFrame(index=out.keys())
for i, r in enumerate(out.keys()):
    df = out[r]
    cdf.loc[r, 'Commited'] = df.run_output_rdn_2000.iloc[-1] / df.run_output_rdn_2000.iloc[0] 
cdf.sort_values(by='Commited')

## Drift 

In [ ]:
ptdf = tdf / tdf.iloc[0]
ptdf.plot();

In [ ]:
ptdf.loc[[0, 100, 300]]